In [12]:
import glob
from os import path
import json
import numpy as np
from scipy.stats import sem
import pandas as pd

In [13]:
abl_fp = "/data/b2p-siteident/experiments/new_baseline_mods"

In [14]:
header = [
    "data_mod", "version", "freeze", "use_several_samples", "tile_size", 
    "pretrained"
]
metrics = [
    # "val_acc", "val_weighted_f1",
    "test_acc",
    # "test_weighted_f1",
    # # "test_rw_acc",
    # "test_rw_weighted_f1",
    # "test_ug_acc", 
    # "test_ug_weighted_f1",
    "ecount"
]

entries_dict = {}
for exp_fp in sorted(glob.glob(path.join(abl_fp, "*"))):
    if not (path.isfile(path.join(exp_fp, "stats_True_32.json")) and
            path.isfile(path.join(exp_fp, "stats_False_32.json"))):
            print("{} not finished.".format(exp_fp))
            continue
    with open(path.join(exp_fp, "opts.json")) as f:
        opts = json.load(f)
    
    with open(path.join(exp_fp, "stats_True_32.json")) as f:
        stats_no_several = json.load(f)
    
    with open(path.join(exp_fp, "stats_False_32.json")) as f:
        stats_several = json.load(f)
    
    data_mod = "small" if len(opts["data_modalities"]) < 7 else "large"
    data_version = opts["data_version"]
    freeze = opts["use_last_n_layers"]
    tile_size = opts["tile_size"]
    pretrained = not opts["no_use_pretrained"]
    for use_several in [True, False]:
        if use_several:
            stats = stats_several
        else:
            stats = stats_no_several
        key = (data_mod, data_version, freeze, use_several, tile_size, pretrained)
        if key not in entries_dict:
            entries_dict[key] = {m: [] for m in metrics}
            entries_dict[key]["ecount"] = 0
            entries_dict[key]["path"] = exp_fp
        for m in metrics:
            if m == "ecount":
                continue
            entries_dict[key][m].append(stats[m])
        entries_dict[key]["ecount"] += 1

entries = []
i = 0
for k, entry in entries_dict.items():
    avg_entry = list(k)
    for m in metrics:
        if m == "ecount":
            avg_entry.append(entry[m])
        else:
            val = np.array(entry[m])
            if "acc" in m:
                val *= 100
            avg_entry.append(round(np.mean(val), 2))
            avg_entry.append(round(sem(val), 2))
    entries.append(avg_entry)
    i += 1

metrics_header = []
for m in metrics:
    if m == "ecount":
        metrics_header.append(m)
    else:
        metrics_header.append(m + "_m")
        metrics_header.append(m + "_ste")
df = pd.DataFrame(entries, columns=header + metrics_header)

In [18]:
df[
    (df.tile_size == 300) 
    # & 
    # (df.use_several_samples == True) & 
    # (df.freeze == -1)
].sort_values(
    by=["version", "data_mod", "tile_size" , "freeze"])

,data_mod,version,freeze,use_several_samples,tile_size,pretrained,test_acc_m,test_acc_ste,ecount
16,large,v1,-1,True,300,True,77.34,0.05,3
17,large,v1,-1,False,300,True,72.10,0.63,3
18,large,v1,9,True,300,True,73.95,0.28,3
19,large,v1,9,False,300,True,69.46,0.44,3
20,small,v1,-1,True,300,True,78.20,0.12,3
21,small,v1,-1,False,300,True,73.52,0.56,3
22,small,v1,9,True,300,True,76.13,0.72,3
23,small,v1,9,False,300,True,70.97,0.47,3
24,large,v2,-1,True,300,True,83.45,0.95,3
25,large,v2,-1,False,300,True,78.96,0.62,3


\res{0.58}{0.02} & \res{0.66}{<0.01} & \res{0.68}{0.01} & & \res{0.53}{0.03} & \res{0.71}{0.01} & \res{0.74}{0.01}

\res{0.78}{0.01} & \res{0.66}{0.01} & \res{0.75}{0.01} & & \res{0.72}{0.03} & \res{0.83}{0.01} & \res{0.82}{0.01}

\res{0.78}{<0.01} & \res{0.63}{0.01} & \res{0.74}{<0.01} & \res{0.79}{0.01} & \res{0.77}{0.02} & \res{0.83}{0.01}